In [ ]:
# @title Copyright & License (click to expand)
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 自定义表格模型的顶点AI模型监控

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_monitoring/get_started_with_model_monitoring_custom.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> 在Colab中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Fmodel_monitoring%2Fget_started_with_model_monitoring_custom.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> 在Colab企业版中打开
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/model_monitoring/get_started_with_model_monitoring_custom.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> 在Workbench中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_monitoring/get_started_with_model_monitoring_custom.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> 在GitHub上查看
    </a>
  </td>
</table>

## 概述

本教程演示了如何针对自定义表格模型使用 Vertex AI Model Monitoring。

了解更多关于[Vertex AI Model Monitoring](https://cloud.google.com/vertex-ai/docs/model-monitoring)。

### 目标

在这个笔记本中，您将学习如何使用Vertex AI Model Monitoring服务来检测自定义表格模型中输入预测请求中的特征倾斜和漂移。

本教程使用以下Vertex AI服务：

- Vertex AI Model Monitoring
- Vertex AI Prediction
- Vertex AI模型资源
- Vertex AI端点资源

执行的步骤包括：

- 下载预训练的自定义表格模型。
- 将预训练模型上传到Vertex AI模型注册表。
- 将模型资源部署到Vertex AI端点资源。
- 配置端点资源进行模型监控。
- 生成合成预测请求以模拟倾斜。
- 等待电子邮件提醒通知。
- 生成合成预测请求以模拟漂移。
- 等待电子邮件提醒通知。

了解更多关于[Vertex AI Model Monitoring](https://cloud.google.com/vertex-ai/docs/model-monitoring/overview)。

### 模型

本教程使用一个预训练模型，模型工件存储在一个公共云存储桶中。

该模型基于一篇关于流失预测模型的[博客文章](https://cloud.google.com/blog/topics/developers-practitioners/churn-prediction-game-developers-using-google-analytics-4-ga4-and-bigquery-ml)。这个模型涉及大量的日志数据，描述了游戏用户与网站的互动方式。原始数据包含以下信息类别：

- 身份 - 独特的玩家标识号。
- 人口特征 - 有关玩家的信息，例如玩家所在地理区域。
- 行为特征 - 玩家触发某些游戏事件的次数，例如达到新等级的次数。
- 流失倾向 - 这是标签或目标特征。它提供了一个估计的概率，即这个玩家可能会流失，即不再是活跃玩家。

### 成本

此教程使用 Google Cloud 的计费组件：

* Vertex AI
* BigQuery
* Cloud Storage

了解 [Vertext AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用量生成成本估算。

开始使用

### 为 Python 安装 Vertex AI SDK 和其他所需的包

In [ ]:
# Install required packages.
! pip3 install --quiet --upgrade google-cloud-aiplatform \
                                 google-cloud-bigquery

重启运行时（仅限Colab）

为了使用新安装的包，你必须在Google Colab上重新启动运行时。

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ 内核即将重新启动。请等待完成后再继续下一步。⚠️</b>
</div>

### 验证您的笔记本环境（仅限Colab）

在Google Colab上验证您的环境。

In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

设置Google Cloud项目信息

了解有关[设置项目和开发环境](https://cloud.google.com/vertex-ai/docs/start/cloud-environment)的更多信息。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

用户电子邮件

将您的用户电子邮件地址设置为接收监控警报。

In [ ]:
import os

USER_EMAIL = "[your-email-address]"  # @param {type:"string"}

if os.getenv("IS_TESTING"):
    USER_EMAIL = "noreply@google.com"

关于服务账户和权限的注意事项

**默认情况下不需要配置**，如果遇到任何权限相关问题，请确保上述服务账户具有所需的角色：

|服务账户邮箱|描述|角色|
|---|---|---|
|PROJECT_NUMBER-compute@developer.gserviceaccount.com|Compute Engine 默认服务账户|Dataflow Admin、Dataflow Worker、Storage Admin、BigQuery Admin、Vertex AI User|
|service-PROJECT_NUMBER@gcp-sa-aiplatform.iam.gserviceaccount.com|AI Platform 服务代理|Vertex AI 服务代理|


1. 进入[IAM控制台](https://console.cloud.google.com/iam-admin/iam)。
2. 勾选**包括 Google 提供的角色授予**复选框。
3. 找到上述邮箱。
4. 授予相应的角色。

使用来自不同项目的数据源：

- 对于BigQuery数据源，请为两个服务账户授予“BigQuery数据查看器”角色。
- 对于CSV数据源，请为两个服务账户授予“存储对象查看器”角色。

创建一个云存储桶

创建一个存储桶，用于存储中间产物，如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

如果您的存储桶还不存在：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $LOCATION -p $PROJECT_ID $BUCKET_URI

### 导入库

In [ ]:
import google.cloud.aiplatform as aiplatform
from google.cloud import bigquery
from google.cloud.aiplatform import model_monitoring

### 初始化 Python 的 Vertex AI SDK

要开始使用 Vertex AI，您必须[启用 Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

### 创建 BigQuery 客户端

在本教程中，您将使用与用于训练预训练模型相同的公共 BigQuery 表中的数据。您将创建一个客户端接口，随后用于访问数据。

In [ ]:
bqclient = bigquery.Client(project=PROJECT_ID)

设置硬件加速器

您可以为预测设置硬件加速器（例如，GPU），或者仅使用 CPU。硬件加速器可以降低预测请求的延迟响应时间。在选择硬件加速器时，要考虑延迟与额外成本之间的平衡。

将变量`DEPLOY_GPU/DEPLOY_NGPU`设置为使用支持 GPU 的容器映像以及分配给虚拟机（VM）实例的 GPU 数量。例如，要使用一张包含 4 个 Nvidia Tesla T4 GPU 的 GPU 容器映像分配给每个 VM，您可以指定：

    (aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_T4, 4)

查看[可用加速器的位置](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)。

否则，请指定`(None, None)`以使用一个在 CPU 上运行的容器映像。

In [ ]:
GPU = False
if GPU:
    DEPLOY_GPU, DEPLOY_NGPU = (aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_T4, 1)
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

设置预构建的容器

设置用于预测的预构建Docker容器映像。

有关最新列表，请参阅[预构建的预测容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
if GPU:
    DEPLOY_VERSION = "tf2-gpu.2-5"
else:
    DEPLOY_VERSION = "tf2-cpu.2-5"

DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    LOCATION.split("-")[0], DEPLOY_VERSION
)

print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

设置机器类型

接下来，设置用于训练和预测的机器类型。

- 设置变量 `DEPLOY_COMPUTE` 来配置预测时的计算资源。
 - 设置一个 `机器类型`:
     - `n1-standard`: 每个 vCPU 3.75GB 内存
     - `n1-highmem`: 每个 vCPU 6.5GB 内存
     - `n1-highcpu`: 每个 vCPU 0.9GB 内存
 - `vCPUs`: \[2, 4, 8, 16, 32, 64, 96\] 中的数字

**注意**：您可以使用 n2 和 e2 机器类型进行训练和部署，但它们不支持 GPU。

In [ ]:
MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

## Vertex AI模型监控简介

Vertex AI模型监控支持自动化机器学习表格模型和自定义表格模型。您可以监控传入预测请求中特征的倾斜和漂移检测，或者监控传出预测响应中特征的归因（可解释AI）。换句话说，您监控衡量特征对输出（预测）的贡献的归因的分布。

以下是启用模型监控的基本步骤：

1. 部署Vertex AI自动化机器学习或自定义表格模型到Vertex AI端点。
2. 配置模型监控规范。
3. 将模型监控规范上传到Vertex AI端点。
4. 上传模式或使用*输入模式*的自动生成进行解析。
5. 对于特征倾斜检测，上传训练数据。这样可以自动生成特征分布。
6. 对于特征归因，上传相应的Vertex Explainable AI规范。

配置完成后，您可以启用/禁用监控，更改警报并更新模型监控配置。

启用模型监控后，采样的传入预测请求将被记录到BigQuery表中。然后分析记录请求中包含的输入特征值，以便定期检查倾斜或漂移。您设置一个采样率来监控模型的一部分生产输入，并设置监控间隔。

模型监控服务需要知道如何解析特征值，这称为输入模式。对于自动化机器学习表格模型，输入模式会自动生成。对于自定义表格模型，服务会尝试从前1000个预测请求中自动推导输入模式。另外，您也可以上传输入模式。

对于倾斜检测，监控服务需要训练数据中值的统计分布基线。对于自动化机器学习表格模型，这是自动生成的。对于自定义表格模型，您将训练数据上传到服务中，并让服务自动推导分布。

对于特征归因中的倾斜和漂移检测，您需要为已部署的自定义表格模型启用Vertex Explainable AI功能。对于自动化机器学习模型，Vertex Explainable AI会自动启用。

了解更多关于[Vertex AI模型监控简介](https://cloud.google.com/vertex-ai/docs/model-monitoring/overview)。

### 将模型工件上传到Vertex AI模型注册表

首先，使用`upload()`方法将预训练的自定义表格模型工件作为Vertex AI模型资源上传，使用以下参数：

- `display_name`：模型资源的人类可读名称。
- `artifact_uri`：模型工件的Cloud Storage位置。
- `serving_container_image`：模型部署到Vertex AI端点资源时要使用的服务容器映像。
- `sync`：是否等待进程完成，还是立即返回（异步）。

In [ ]:
MODEL_ARTIFACT_URI = "gs://mco-mm/churn"

model = aiplatform.Model.upload(
    display_name="churn",
    artifact_uri=MODEL_ARTIFACT_URI,
    serving_container_image_uri=DEPLOY_IMAGE,
    sync=True,
)

print(model)

### 部署模型到终端点

接下来，使用`deploy()`方法，将您的Vertex AI模型资源部署到一个Vertex AI终端点资源，参数如下：

- `deploy_model_display`: 部署模型的人类可读名称。
- `machine_type`: 每个VM节点实例的机器类型。
- `min_replica_count`: 为自动扩展预留的节点的最小数量。
- `max_replica_count`: 为自动扩展预留的节点的最大数量。
- `accelerator_type`: 每个预留节点的GPU加速器类型（如果有）。
- `accelerator_count`: 每个预留节点的GPU加速器数量（如果有）。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

if GPU:
    endpoint = model.deploy(
        deployed_model_display_name="churn",
        machine_type=DEPLOY_COMPUTE,
        min_replica_count=MIN_NODES,
        max_replica_count=MAX_NODES,
        accelerator_type=DEPLOY_GPU.name,
        accelerator_count=DEPLOY_NGPU,
    )
else:
    endpoint = model.deploy(
        deployed_model_display_name="churn",
        machine_type=DEPLOY_COMPUTE,
        min_replica_count=MIN_NODES,
        max_replica_count=MAX_NODES,
    )

## 配置监控作业

配置监控作业包括以下规范：

- `alert_config`：应该接收监控警报的电子邮件地址。
- `schedule_config`：分析预测的时间窗口。
- `logging_sampling_strategy`：对预测请求进行采样的速率。
- `drift_config`：要监视的特征和漂移阈值。
- `skew_config`：要监视的特征和偏斜阈值。

### 配置警报规范

使用以下设置配置`alerting_config`规范：

- `user_emails`：应接收警报的一个或多个电子邮件地址列表。
- `enable_logging`：将检测到的异常流式传输到云日志。默认值为False。

In [ ]:
# Create alerting configuration.
alerting_config = model_monitoring.EmailAlertConfig(
    user_emails=[USER_EMAIL], enable_logging=True
)

### 配置监控间隔规范

接下来，您将使用以下设置配置`schedule_config` 规范：

- `monitor_interval`: 设置模型监控作业的调度间隔时间（小时）。最小时间间隔为1小时。

In [ ]:
# Monitoring Interval
MONITOR_INTERVAL = 1  # @param {type:"number"}

# Create schedule configuration
schedule_config = model_monitoring.ScheduleConfig(monitor_interval=MONITOR_INTERVAL)

### 配置采样规范

现在，您需要使用以下设置来配置 `logging_sampling_strategy` 规范：

- `sample_rate`: 以百分比形式（在 0 到 1 之间）设置随机抽样预测请求以进行监控。所选样本将被记录到 BigQuery 表中。

In [ ]:
# Sampling rate (optional, default=.8)
SAMPLE_RATE = 0.5  # @param {type:"number"}

# Create sampling configuration
logging_sampling_strategy = model_monitoring.RandomSampleConfig(sample_rate=SAMPLE_RATE)

### 配置漂移检测规范

然后，您可以使用以下设置配置`drift_config`规范：

- `drift_thresholds`：一个键/值对字典，其中键是用于监视漂移的输入特征，值表示检测阈值。如果未指定，则特征的默认漂移阈值为0.3（30%）。

**注意：** 启用漂移检测是可选的。

In [ ]:
DRIFT_THRESHOLD_VALUE = 0.05

DRIFT_THRESHOLDS = {
    "country": DRIFT_THRESHOLD_VALUE,
    "cnt_user_engagement": DRIFT_THRESHOLD_VALUE,
}

drift_config = model_monitoring.DriftDetectionConfig(drift_thresholds=DRIFT_THRESHOLDS)

### 配置偏斜检测规范

接下来，您将使用以下设置配置`skew_config`规范：

- `data_source`：原始训练数据集的数据源。数据源的格式默认为 BigQuery 表。否则，必须将设置`data_format`设置为以下值之一。数据的位置必须是 Cloud Storage 位置。
  - `csv`
  - `jsonl`
  - `tf-record`
- `skew_thresholds`：一个键/值对的字典，其中键是监控数据倾斜的输入特征。值表示检测阈值。如果未指定，则特征的默认倾斜阈值为 0.3（30%）。
- `target_field`：训练数据集的目标标签

**注意：**启用偏斜检测是可选的。

In [ ]:
# URI to training dataset.
DATASET_BQ_URI = "bq://mco-mm.bqmlga4.train"  # @param {type:"string"}
# Prediction target column name in training dataset.
TARGET = "churned"

SKEW_THRESHOLD_VALUE = 0.5

SKEW_THRESHOLDS = {
    "country": SKEW_THRESHOLD_VALUE,
    "cnt_user_engagement": SKEW_THRESHOLD_VALUE,
}

skew_config = model_monitoring.SkewDetectionConfig(
    data_source=DATASET_BQ_URI, skew_thresholds=SKEW_THRESHOLDS, target_field=TARGET
)

### 组装目标规范

最后，您将使用以下设置将目标规范（`objective_config`）组装起来:

- `skew_detection_config`：（可选）用于倾斜检测配置的规范。
- `drift_detection_config`：（可选）用于漂移检测配置的规范。
- `explanation_config`：（可选）在启用特征归因监控时用于解释的规范。

In [ ]:
objective_config = model_monitoring.ObjectiveConfig(
    skew_detection_config=skew_config,
    drift_detection_config=drift_config,
    explanation_config=None,
)

### 创建输入模式

监控服务需要了解模型的特征和数据类型以及模型的特征输入，这被称为*输入模式*。*输入模式*可以是:
- 预加载到监控服务中。
- 在接收到前1000个预测实例后，由监控服务自动生成。

在本教程中，您将预加载*输入模式*。

#### 创建预定义输入模式

预定义的*输入模式*被指定为一个YAML文件。在本例中，您检索用于训练数据的BigQuery模式，其中包括特征名称和数据类型，以生成YAML规范。预定义的*输入模式*必须加载到云存储位置。

了解更多关于[用于解析输入的自定义实例模式](https://cloud.google.com/vertex-ai/docs/model-monitoring/overview#custom-input-schemas)。

In [ ]:
# Get the BQ table

table = bigquery.TableReference.from_string(DATASET_BQ_URI[5:])
bq_table = bqclient.get_table(table)

yaml = """type: object
properties:
"""

schema = bq_table.schema
for feature in schema:
    if feature.name == TARGET:
        continue
    if feature.field_type == "STRING":
        f_type = "string"
    else:
        f_type = "integer"
    yaml += f"""  {feature.name}:
    type: {f_type}
"""

yaml += """required:
"""
for feature in schema:
    if feature.name == TARGET:
        continue
    yaml += f"""- {feature.name}
"""

print(yaml)

with open("schema.yaml", "w") as f:
    f.write(yaml)

! gsutil cp schema.yaml {BUCKET_URI}/schema.yaml

### 创建监控任务

使用Vertex AI的[`ModelDeploymentMonitoringJob.create()`](https://cloud.google.com/python/docs/reference/aiplatform/1.48.0/summary_method#google_cloud_aiplatform_ModelDeploymentMonitoringJob_create_summary)方法，根据您的监控规范创建一个监控任务，具有以下参数：

- `display_name`：监控任务的可读名称。
- `project`：项目ID。
- `region`：区域。
- `endpoint`：要启用监控的Vertex AI端点的完全限定资源名称。
- `logging_sampling_strategy`：采样配置的规范。
- `schedule_config`：调度配置的规范。
- `alert_config`：警报配置的规范。
- `objective_configs`：目标配置的规范。
- `analysis_instance_schema_uri`：包含*输入模式*的YAML文件的位置。

In [ ]:
monitoring_job = aiplatform.ModelDeploymentMonitoringJob.create(
    display_name="churn",
    project=PROJECT_ID,
    location=LOCATION,
    endpoint=endpoint,
    logging_sampling_strategy=logging_sampling_strategy,
    schedule_config=schedule_config,
    alert_config=alerting_config,
    objective_configs=objective_config,
    analysis_instance_schema_uri=f"{BUCKET_URI}/schema.yaml",
)

print(monitoring_job)

#### 监控工作的电子邮件通知。

电子邮件通知将发送到警报配置中的电子邮件地址，通知模型监控工作现已启用。

电子邮件的内容如下：

您好，Vertex AI 客户，

您收到此邮件是因为您正在使用 Vertex AI 模型监控服务。
此邮件是为了通知您，我们收到了您设置漂移或偏差检测的请求，针对以下列出的预测终点。从现在开始，传入的预测请求将被抽样并记录用于分析。
原始请求和响应将从预测服务中收集，并保存在 bq://[your-project-id].model_deployment_monitoring_[endpoint-id].serving_predict。

#### 监控作业状态

在您启动 Vertex AI 模型监控作业之后，它会保持在 **PENDING** 状态，直到计算出 `偏斜分布基线` 为止。监控服务会启动一个批处理作业，从训练数据中生成分布基线。

一旦基线分布生成完成，监控作业就会变为 **OFFLINE** 状态。按照每个间隔基础，例如，每小时一次，监控作业会进入 **RUNNING** 状态并分析采样数据。一旦完成，它会回到 **OFFLINE** 状态，并等待下一个预定的分析。

In [ ]:
jobs = monitoring_job.list(filter="display_name=churn")
job = jobs[0]
print(job.state)

### 基线分布的自动生成

接下来，监控服务会创建一个批处理作业来分析训练数据以生成基线分布。一旦完成，监控服务会开始在指定的时间间隔进行监控。

In [ ]:
import time

# Pause a bit for the baseline distribution to be calculated
if os.getenv("IS_TESTING"):
    time.sleep(180)

生成用于检测偏斜的合成预测请求

现在，从BigQuery训练表中提取前1000个实例以创建预测请求。然后，通过进行以下更新修改数据（合成数据），以在预测请求中触发偏斜检测：

- `country`：将所有值设为加拿大。

In [ ]:
# Download the table.
table = bigquery.TableReference.from_string(DATASET_BQ_URI[5:])

rows = bqclient.list_rows(table, max_results=1000)

instances = []
for row in rows:
    instance = {}
    for key, value in row.items():
        if key == TARGET:
            continue
        if value is None:
            value = ""
        if key == "country":
            value = "Canada"
        instance[key] = value
    instances.append(instance)

print(len(instances))

### 发送预测请求

接下来，您可以使用`predict()`方法向您的Vertex AI端点资源发送1000个预测请求。

In [ ]:
for instance in instances:
    response = endpoint.predict(instances=[instance])

prediction = response[0]

# print the prediction for the first instance
print(prediction[0])

### 记录抽样请求

监控服务启动后，抽样预测请求将被记录到云存储中。在下一个监控间隔内，抽样预测将被复制到BigQuery日志表中。一旦条目被记录，监控服务将分析抽样数据。

接下来，您需要等待第一批被记录的条目出现在BigQuery日志表中，用于预测样本。由于您发送了1000个预测请求，50%抽样率，您应该看到大约500条记录。

In [ ]:
while True:
    time.sleep(180)

    ENDPOINT_ID = endpoint.resource_name.split("/")[-1]

    table = bigquery.TableReference.from_string(
        f"{PROJECT_ID}.model_deployment_monitoring_{ENDPOINT_ID}.serving_predict"
    )
    rows = bqclient.list_rows(table)
    print(rows.total_rows)
    if rows.total_rows > 0:
        break

### 监控期间的偏斜检测

特征输入的偏斜检测将在下一个监控间隔期间进行。在本教程中，您将监控间隔设置为一小时。因此，大约一个小时后，您的监控作业从 **离线** 状态变为 **运行** 状态。在运行时，它会分析该间隔期间的预测中记录的采样表，并将它们与基准分布进行比较。

分析完成后，监控作业会发送电子邮件通知，告知检测到的偏斜，本例中为 `country`。此后，监控作业将进入 **离线** 状态，直到下一个间隔期间。

#### 等待监控间隔

从监控间隔的分析完成时刻到收到电子邮件警报可能需要 40 分钟或更长时间。

电子邮件的内容如下：

<blockquote>
   你好，Vertex AI 用户，

您收到此邮件是因为您订阅了 Vertex AI 模型监测服务。
此邮件仅用于通知您，您部署的模型中检测到一些异常情况，并可能需要您关注。

基本信息：

端点名称：projects/[your-project-id]/locations/us-central1/endpoints/3315907167046860800
监控作业：projects/[your-project-id]/locations/us-central1/modelDeploymentMonitoringJobs/8672170640054157312
统计数据和异常根路径（Google Cloud 存储）：gs://cloud-ai-platform-773884b1-2a32-48d6-8b83-c03cde416b68/model_monitoring/job-8672170640054157312
BigQuery 命令：SELECT * FROM `bq://[your-project-id].model_deployment_monitoring_3315907167046860800.serving_predict`

训练预测偏斜异常（原始特征）：

异常报告路径（Google Cloud 存储）：gs://cloud-ai-platform-773884b1-2a32-48d6-8b83-c03cde416b68/model_monitoring/job-8672170640054157312/serving/2022-08-25T00:00/stats_and_anomalies/<deployed-model-id>/anomalies/training_prediction_skew_anomalies

有关警报的更多信息，请访问模型监测警报页面。

部署的模型 id: <deployed-model-id>

特征名称	异常简要描述	异常详细描述
country	训练和服务之间的 Linfty 距离较大	Linfty 距离介于训练和服务之间，为 0.947563（至多六个有效数字），高于阈值 0.5。差异最大的特征值为：Canada
<blockquote>

In [ ]:
if os.getenv("IS_TESTING"):
    time.sleep(60 * 45)

### 生成用于漂移检测的合成预测请求

接下来，您从BigQuery训练表中提取相同的前1000个实例用于预测请求。然后，通过进行以下更新修改数据（合成），以触发预测请求中的漂移检测：

- `cnt_user_engagement`：将值增加4倍。

In [ ]:
# Download the table.
table = bigquery.TableReference.from_string(DATASET_BQ_URI[5:])

rows = bqclient.list_rows(table, max_results=1000)

instances = []
for row in rows:
    instance = {}
    for key, value in row.items():
        if key == TARGET:
            continue
        if value is None:
            value = ""
        elif key == "cnt_user_engagement":
            value = int(value * 4)
        instance[key] = value
    instances.append(instance)

print(len(instances))

进行预测请求

接下来，您可以使用`predict()`方法将1000个预测请求发送至您的Vertex AI端点资源。

In [ ]:
for instance in instances:
    response = endpoint.predict(instances=[instance])

prediction = response[0]

# print the prediction for the first instance
print(prediction[0])

### 记录抽样请求

在下一个监测间隔中，抽样的预测结果会被复制到BigQuery的日志表中。一旦条目被记录，监测服务就会分析抽样数据。

接下来，您等待第一个被记录的条目出现在BigQuery的日志表中用于预测样本。由于您发送了1000个预测请求，抽样率为50%，您应该会看到大约1000个条目。

In [ ]:
while True:
    time.sleep(180)

    ENDPOINT_ID = endpoint.resource_name.split("/")[-1]

    table = bigquery.TableReference.from_string(
        f"{PROJECT_ID}.model_deployment_monitoring_{ENDPOINT_ID}.serving_predict"
    )
    rows = bqclient.list_rows(table)
    print(rows.total_rows)
    if rows.total_rows > 550:
        break

### 监控过程中的漂移检测

特征输入的漂移检测发生在下一个监控间隔。在本教程中，您将监控间隔设置为一小时。因此，大约一个小时后，您的监控作业从**离线**状态变为**运行**状态。在运行过程中，它会分析此间隔内预测的日志样本表，并将其与上一个监控间隔的分布进行比较。

一旦分析完成，监控作业会通过电子邮件通知您检测到的漂移情况，本例中为`cnt_user_engagement`。然后，监控作业会进入**离线**状态，直到下一个间隔。

#### 等待监控间隔

从监控间隔分析完成的时刻到您收到电子邮件警报可能需要40分钟或更长时间。

In [ ]:
if os.getenv("IS_TESTING"):
    time.sleep(60 * 45)

### 删除监控任务

收到电子邮件警报并验证内容后，你可以：
- 使用`pause()`方法暂停监控任务。
- 使用`delete()`方法删除监控任务。

In [ ]:
# Pause the job
monitoring_job.pause()
# Delete the job
monitoring_job.delete()

清理

要清理本项目中使用的所有Google Cloud资源，您可以删除用于教程的[Google Cloud
项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源。

In [ ]:
# Undeploy the model from endpoint before deletion
endpoint.undeploy_all()

# Delete the endpoint
endpoint.delete()

# Delete the model
model.delete()

# Delete the Cloud Storage bucket
delete_bucket = False  # Set True for deletion
if delete_bucket:
    ! gsutil rm -rf {BUCKET_URI}

# Delete the locally generated files
! rm -f schema.yaml

# Delete the BigQuery table
! bq rm -f {PROJECT_ID}.model_deployment_monitoring_{ENDPOINT_ID}